# Welcome to Passion!
Passion is a model that can detection anomaly using different methods (Both supervised and unsupervised) <br>

1. The goal for this project is to study the difference between different anomnaly detection model, and to   <br>
 find the state of art method for detecting anomaly in real world data<br>
2. Evaluate the results based on this :real server data+ https://www.kaggle.com/sohier/30-years-of-european-wind-generation (Unsupervised)+ https://github.com/numenta/NAB (Unsupervised+Supervised)  https://www.cs.ucr.edu/~eamonn/time_series_data/ (Supervised) <br>
3. Also use real data generated from server.
4. The model has the following fuctions: <br>
    a. Visualize the input data. Help the user to find critical features within the inputs. <br>
    b. Give user options to choose different models that are suitable for different circumstances. <br>
    c. Evaluate the performance based on the rules in this link https://github.com/numenta/NAB <br>
    d. Save model. Easy to be appplied to other dataset. <br>

5. Add un-labeled and labeled data





# What's new in version 6.1
1. Apply open dataset: https://www.cs.ucr.edu/~eamonn/time_series_data/ (Supervised)
2. Brain-teaser

In [1]:
# import packages 


from matplotlib.pylab import rc
import torch
from scipy.stats import chisquare
from scipy.stats import pearsonr 
import pickle
import pandas as pd
import datetime
import matplotlib
import tensorflow as tf
import sklearn
import math
import matplotlib.pyplot as plt
import xgboost
from xgboost import XGBClassifier
from xgboost import plot_importance
import numpy as np
from sklearn.model_selection import train_test_split 
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder 
import copy
import scipy
import datetime
import time
import os
from sklearn.model_selection import KFold 
from sklearn.metrics import roc_curve 
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
import gc
import json
plot_path = "plots/"



In [2]:
# labeled data:
root_path2 = "Data/Ant_labeled/"

today = []
history = []
label = []
count=0
with open(root_path2+"train_data.txt") as f:
    for line in f:
        temp = json.loads(line)
        today.append(temp["today"])
        history.append(temp["history"])
        label.append(temp["label"])
        count+=1
today = np.array(today)
history = np.array(history)
label = np.array(label).ravel()

In [4]:
# Three classes 1 2 and 3
root_path = "Data/UCRArchive_2018/ChlorineConcentration/"
os.listdir(root_path)

['ChlorineConcentration_TEST.tsv',
 'ChlorineConcentration_TRAIN.tsv',
 'README.md']

In [67]:
train = np.genfromtxt(root_path+"ChlorineConcentration_TRAIN.tsv")
test = np.genfromtxt(root_path+"ChlorineConcentration_TEST.tsv")
X_train, X_test, y_train, y_test = train[:,1:],test[:,1:],train[:,0],test[:,0]

In [72]:


params={}
params['booster'] = "gbtree"
params['gpu_id'] = 0
params['max_bin'] = 512
params['tree_method'] = 'gpu_hist'
"""

params['learning_rate'] = 0.01
params['max_depth'] = 12

params['reg_alpha'] = 0.15
params['reg_lamdba'] = 0.85



"""
model = XGBClassifier(n_estimators=1000,n_jobs=-1,**params)
model.fit(X_train,y_train)





XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_bin=512, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
              n_estimators=1000, n_jobs=-1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [73]:
Y_predict_test = model.predict(X_test)
y_pred = Y_predict_test
Y_predict_test_xgboost = y_pred
mask_good = abs(Y_predict_test-y_test)<0.01

print("Good=%d Bad=%d"%(len(Y_predict_test[mask_good]),len(Y_predict_test)-len(Y_predict_test[mask_good])))

print("Accuracy=%.4f for testing set"%(len(Y_predict_test[mask_good])/len(Y_predict_test)))



Good=3038 Bad=802
Accuracy=0.7911 for testing set


In [74]:
# AUROC for multi-classes
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
y_prob = model.predict_proba(X_test)
macro_roc_auc_ovo = roc_auc_score(y_test, y_prob, multi_class="ovo",
                                  average="macro")
weighted_roc_auc_ovo = roc_auc_score(y_test, y_prob, multi_class="ovo",
                                     average="weighted")
macro_roc_auc_ovr = roc_auc_score(y_test, y_prob, multi_class="ovr",
                                  average="macro")
weighted_roc_auc_ovr = roc_auc_score(y_test, y_prob, multi_class="ovr",
                                     average="weighted")
print("One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
      "(weighted by prevalence)"
      .format(macro_roc_auc_ovo, weighted_roc_auc_ovo))
print("One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
      "(weighted by prevalence)"
      .format(macro_roc_auc_ovr, weighted_roc_auc_ovr))

One-vs-One ROC AUC scores:
0.904898 (macro),
0.907460 (weighted by prevalence)
One-vs-Rest ROC AUC scores:
0.908243 (macro),
0.909406 (weighted by prevalence)


In [99]:
# Add a diff dimension

X_train, X_test, y_train, y_test = train[:,1:],test[:,1:],train[:,0],test[:,0]
mean = np.nanmedian(np.c_[X_train.T,X_test.T],axis=1)
X_train = np.c_[X_train,X_train-np.tile(mean,(X_train.shape[0],1))]
X_test = np.c_[X_test,X_test-np.tile(mean,(X_test.shape[0],1))]

In [101]:
# Doesn't seem to help...

params={}
params['booster'] = "gbtree"
params['gpu_id'] = 0
params['max_bin'] = 512
params['tree_method'] = 'gpu_hist'
"""

params['learning_rate'] = 0.01
params['max_depth'] = 12

params['reg_alpha'] = 0.15
params['reg_lamdba'] = 0.85



"""
model = XGBClassifier(n_estimators=1000,n_jobs=-1,**params)
model.fit(X_train,y_train)

Y_predict_test = model.predict(X_test)
y_pred = Y_predict_test
Y_predict_test_xgboost = y_pred
mask_good = abs(Y_predict_test-y_test)<0.01

print("Good=%d Bad=%d"%(len(Y_predict_test[mask_good]),len(Y_predict_test)-len(Y_predict_test[mask_good])))

print("Accuracy=%.4f for testing set"%(len(Y_predict_test[mask_good])/len(Y_predict_test)))







Good=3027 Bad=813
Accuracy=0.7883 for testing set


In [102]:
# AUROC for multi-classes
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
y_prob = model.predict_proba(X_test)
macro_roc_auc_ovo = roc_auc_score(y_test, y_prob, multi_class="ovo",
                                  average="macro")
weighted_roc_auc_ovo = roc_auc_score(y_test, y_prob, multi_class="ovo",
                                     average="weighted")
macro_roc_auc_ovr = roc_auc_score(y_test, y_prob, multi_class="ovr",
                                  average="macro")
weighted_roc_auc_ovr = roc_auc_score(y_test, y_prob, multi_class="ovr",
                                     average="weighted")
print("One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
      "(weighted by prevalence)"
      .format(macro_roc_auc_ovo, weighted_roc_auc_ovo))
print("One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
      "(weighted by prevalence)"
      .format(macro_roc_auc_ovr, weighted_roc_auc_ovr))

One-vs-One ROC AUC scores:
0.903401 (macro),
0.905981 (weighted by prevalence)
One-vs-Rest ROC AUC scores:
0.906682 (macro),
0.908078 (weighted by prevalence)
